# NWB workshop
## Converting to NWB with neuroconv
We will deal with two problems that we think arise often.
The first one is having a type of data that you would like
to convert to an NWB file and the second you already have an
NWB and some data you would like to add to the NWB file.

## Prerequisites
If you were running this on your own machine you would have
to install jupyter, neuroconv package and the data.
However for the workshop these steps are baked in
the provided notebooks so you don't need to anything.

## Problem 1:
We will start with the simplest case scenario. The data that we have
is from a csv file. 

In [ ]:
from neuroconv.datainterfaces import CsvTimeIntervalsInterface
from zoneinfo import ZoneInfo
from pathlib import Path
from datetime import datetime

### Create CSV file if not available

In [ ]:
import random
import pandas as pd
data_dir = Path("../data")
data_dir.mkdir(exist_ok=True)
csv_file_path = data_dir / "mydata.csv"

In [ ]:
if not csv_file_path.exists():
    n = 100
    starts  = [-1]*n
    ends  = [-1]*n
    vals = [-1]*n
    s0 = 0
    for i in range(n):
        starts[i]= s0 + random.randrange(5)
        ends[i] = starts[i] + random.randrange(1,50)
        vals[i] = random.uniform(0,1)
        s0 = ends[i]


    pd.DataFrame({'start_time':starts, 'end_time':ends, 'value':vals }).to_csv(csv_file_path,index=False)

### Create interface and get metadata

In [ ]:
interface = CsvTimeIntervalsInterface(file_path=csv_file_path, verbose=False)

In [ ]:
metadata = interface.get_metadata()
metadata

### Attempt to write NWB file

In [ ]:
nwb_file_path = "csv_file.nwb"
interface.run_conversion(nwbfile_path = nwb_file_path, metadata=metadata)

### Add missing metadata

In [ ]:
session_start_time = datetime(2025,1,10,11,45,0, tzinfo=ZoneInfo("Europe/Paris"))
metadata["NWBFile"]["session_start_time"] = session_start_time

### Create NWB file

In [ ]:
interface.run_conversion(nwbfile_path = nwb_file_path, metadata=metadata)

### Look at the generated NWB file

In [ ]:
from pynwb import NWBHDF5IO

In [ ]:
with NWBHDF5IO(nwb_file_path, mode = 'r') as io:
    nwbfile = io.read()
    print(nwbfile)
    starts = nwbfile.trials['start_time'].data
    print(starts[0:5])